In [2]:
import pandas as pd
from web_scraper import *
from time import sleep
from csv_funcs import *
import csv

Step 1: 
<br>Scrape fbref using list of links

In [ ]:
in_file_path = 'C:/Users/fires/Python Projects/Football Match Modelling/epl_teams.txt' #list of fbref team page links
out_file_path = "C:/Users/fires/Python Projects/Football Match Modelling/tmp.csv"

teams_list = open(in_file_path, 'r').read().splitlines() #path to file of team pages

for team_url in teams_list:
    curr_df = fbref_scrape(team_url)
    sleep(4) #avoid rate limiting (20 requests/min is maximum)
    append_csv(curr_df, out_file_path)

Step 2: <br>
Before this step, I sorted all my sheets by date using alphabetical order of home team as a tie breaker so that they line up <br>
Now I can add the betting odds for 2012-2024. Additionally, modify the table manually to have a column for the season and replace home team names so that they match the away team names.


In [ ]:
#load in dataframe of fbref match data
fb_df = pd.read_csv("C:/Users/fires/Python Projects/Football Match Modelling/tmp.csv")
fb_df['B365H'] = pd.Series(dtype='Float32')
fb_df['B365D'] = pd.Series(dtype='Float32')
fb_df['B365A'] = pd.Series(dtype='Float32')

#array of file paths for betting odds data, list allows me to load only one at a time and save memory
main_path = 'C:/Users/fires/Python Projects/Football Match Modelling/betodds/' #folder path, append names to this to get path
odds_file_names = ('12_13.csv','13_14.csv','14_15.csv','15_16.csv','16_17.csv','17_18.csv', '18_19.csv', '19_20.csv', 
                   '20_21.csv', '21_22.csv', '22_23.csv', '23_24.csv')

'''
this loop merges the odds data onto the appropriate section of the all games sheet
'''
fb_index = 380 #start of the 2012-13 season, we don't need odds for the 2011-12 season
for file in odds_file_names:
    odds_df = pd.read_csv(main_path + file)[['B365H','B365D','B365A']] #load in odds for that season
    stop = fb_index + 380
    fb_df.iloc[fb_index:stop,11:] = odds_df.values
    fb_index = stop

fb_df.to_csv('tmp2.csv', index=False)

Step 4: <br>
Get Premier League table data in order to calculate if a match is significant. No team has ever been relegated or secured the title before March so we'll ignore league tables from before this date and assume matches are equally significant for all teams. Since the set used to remove duplicates is unordered, the output needs to be sorted manually.

In [ ]:
#create a set of all unique match dates for 12/13-23/24
#no team has ever been out of contention before March
match_dates = set()
comp_months = [3,4,5,6,7] #months where games can be insignificant
with open('tmp2.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['season'] != '11/12' : #don't need 11/12 since I'm not modelling using this season
            month = int(row['match_date'][:2].replace('/',''))
            if month in comp_months:
                match_dates.add(row['match_date'])

for date in match_dates:
    date_list = date.split('/')
    sleep(4) #avoid rate limiting on the API
    date_df = prem_table_scrape(date_list)
    append_csv(date_df, 'tables_v2.csv')

Step 3: <br>
Load tables back in. Need to manually change some of the longer team names (ex: Manchester United -> Manchester Utd) because fbref abbreviates more

In [ ]:
#import league tables into dictionary of pandas dataframes for fast lookup
tables = {}

#shifts dates one back. tables are from the END of the date listed so in order to tell if a match is significant we need to know the table from 
#the end of the day before
with open('tables.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader) #skip header
    for line in reader:
        date = line[0]
        if tables != {}: #and the date matches, add the line
            if date in tables:
                tables[date].loc[len(tables[date])] = line[1:]
                continue
        tables[date] = pd.DataFrame([line[1:]], columns=header[1:])

Step 4: <br>
Load match data into class structure and use class methods to write training data file.

In [ ]:
from classes import game, team

teams = []

with open('tmp2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader) #skip header
    for line in reader:
        tmp_home = None
        tmp_away = None
        #find/initialize home team
        for team_obj in teams or []:
            if team_obj.getName() == line[2]: 
                tmp_home = team_obj
        if not tmp_home: #if the team is new, add it to the teams list
            tmp_home = team(line[2])
            teams.append(tmp_home) 
        #find/initialize away team
        for team_obj in teams or []:
            if team_obj.getName() == line[3]:
                tmp_away = team_obj
        if not tmp_away: #if the team is new, add it to the teams list
            tmp_away = team(line[3])
            teams.append(tmp_away) 
        #initialize game
        tmp_game = game(line, tmp_home, tmp_away, tables)
        #add game to both teams gameslist
        tmp_home.addGame(tmp_game)
        tmp_away.addGame(tmp_game)

        #if both teams have more than 5 games in the current season:
        if tmp_home.getNumGames() > 5 and tmp_away.getNumGames() > 5:
            home_prev_results = tmp_home.getPrevMatches()
            away_prev_results = tmp_away.getPrevMatches()
            row = {
                "date" : line[0],
                "season" : line[1],
                "home" : line[2],
                "away" : line[3],
                "result" : line[4],
                "gd" : int(line[5]) - int(line[6]),
                "win_odds" : line[11],
                "draw_odds" : line[12],
                "loss_odds" : line[13],
                "h_prev_result1" : home_prev_results[0],
                "h_prev_result2" : home_prev_results[1],
                "h_prev_result3" : home_prev_results[2],
                "a_prev_result1" : away_prev_results[0],
                "a_prev_result2" : away_prev_results[1],
                "a_prev_result3" : away_prev_results[2],
                "form_diff" : tmp_home.getForm() - tmp_away.getForm(),
                "home_yellows" : tmp_home.getPrevMatchYellow(),
                "away_yellows" : tmp_away.getPrevMatchYellow(),
                "home_reds" : tmp_home.getPrevMatchRed(),
                "away_reds" : tmp_away.getPrevMatchRed(),
                "significance" : tmp_game.significant
            }
            append_row("tmp3.csv", row)
        else:
            #included with missing data so that I can look up previous fixtures
            row = {
                "date" : line[0],
                "season" : line[1],
                "home" : line[2],
                "away" : line[3],
                "result" : line[4],
                "gd" : int(line[5]) - int(line[6]),
                "win_odds" : line[11],
                "draw_odds" : line[12],
                "loss_odds" : line[13],
                "h_prev_result1" : '',
                "h_prev_result2" : '',
                "h_prev_result3" : '',
                "a_prev_result1" : '',
                "a_prev_result2" : '',
                "a_prev_result3" : '',
                "form_diff" : '',
                "home_yellows" : '',
                "away_yellows" : '',
                "home_reds" : '',
                "away_reds" : '',
                "significance" : ''
            }
            append_row("tmp3.csv", row)

Step 5: <br>
Get previous season average attendance for every team. Manually fill in values for teams that were promoted.

In [ ]:
in_file_path = 'C:/Users/fires/Python Projects/Football Match Modelling/epl_teams.txt' #list of fbref team page links
out_file_path = "C:/Users/fires/Python Projects/Football Match Modelling/attendance.csv"

teams_list = open(in_file_path, 'r').read().splitlines() #path to file of team pages

for team_url in teams_list:
    scrape = avg_attendance(team_url)
    append_csv(scrape, out_file_path)
    sleep(4)